In [ ]:
import torch
from transformers import pipeline
import sys
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
dataset_name = "davidgaofc/pairwise_setup"
dataset = load_dataset(dataset_name)

In [ ]:
pairwise_comp = pipeline("text-classification", model="davidgaofc/hh-labeler")

In [ ]:
pairwise_comp("I am happy.")

In [ ]:
import random
def compare(entry, model1, model2):
    if(entry[model1] == None or entry[model2] == None):
        return {'chosen_response': None, 'rejected_response': None}
    if(random.random() < 0.5):
        prompt = entry['Question'] + "\nAssistant:\n" + "1. "+entry[model1] + "\n2. "+entry[model2]
        selected = pairwise_comp(prompt)[0]
        if(selected['label'] == 'LABEL_0'):
            chosen = model1
            rejected = model2
        else:
            chosen = model2
            rejected = model1
    else:
        prompt = entry['Question'] + "\nAssistant:\n" + "1. "+entry[model2] + "\n2. "+entry[model1]
        selected = pairwise_comp(prompt)[0]
        if(selected['label'] == 'LABEL_0'):
            chosen = model2
            rejected = model1
        else:
            chosen = model1
            rejected = model2
    return {'chosen_response': chosen, 'rejected_response': rejected}

In [ ]:
processed_dataset = dataset.map(lambda x: compare(x, "SFT", "Base_PPO"))

In [ ]:
from collections import Counter
chosen_responses = processed_dataset['train']['chosen_response']
label_counts = Counter(chosen_responses)
for label, count in label_counts.items():
    print(f"Label '{label}': {count} occurrences")

In [ ]:
processed_dataset2 = dataset.map(lambda x: compare(x, "SFT", "Prima_PPO"))

In [ ]:
from collections import Counter
chosen_responses = processed_dataset2['train']['chosen_response']
label_counts = Counter(chosen_responses)
for label, count in label_counts.items():
    print(f"Label '{label}': {count} occurrences")

In [ ]:
processed_dataset3 = dataset.map(lambda x: compare(x, "Base_PPO", "Prima_PPO"))

In [ ]:
from collections import Counter
chosen_responses = processed_dataset3['train']['chosen_response']
label_counts = Counter(chosen_responses)
for label, count in label_counts.items():
    print(f"Label '{label}': {count} occurrences")